#  Code for 2D staggered Lagrangian method in Cartersian coordinates

## Control equations

### Momentum equation

$$\tag{1.1}\rho \frac{d\mathbf{v}}{dt} =  -\nabla p $$ 
where $\mathbf{v}$ is the vector of velocity, and $\rho$ is the density and $p$ is the pressure.  

### Energy equation

$$ \rho \frac{dE}{dt} = -\nabla \cdot (p\mathbf{v})$$ 

where $E$ is the specific total energy and it has a realtion with the specific internal energy 
$$E = e + \frac{1}{2}\mathbf{v}\cdot\mathbf{v}$$

By (1.1) and (1.2) we have 
$$\tag{1.3} \rho \frac{de}{dt} = - p\nabla\cdot \mathbf{v}$$

In  a Cartersian coordinates $x-y$, The equations (1.1) and (1.3) can be written as
$$\rho \frac{du}{dt} = -\frac{\partial p}{\partial x},\quad \rho \frac{dv}{dt} = -\frac{\partial p}{\partial y}$$
and
$$\rho \frac{de}{dt} = -p (\frac{\partial u}{\partial x} +\frac{\partial v}{\partial y})$$

## Compatible Staggered Discretization

<img src="Grid.png" width = "800" height = "400" div align=center /> 

<img src="cellpoint.png" width = "800" height = "400" div align=center />

### Quatities:

On point: ($x_p$,$y_p$), ($u_p$,$v_p$)

Zonal:  $V_z$, $\rho_z$, $m_z$, $e_z$, $p_z$

By the relation 
$$\frac{dm_z}{dt} =0$$
we have 
$$\rho_z(t) = m_z/V_z(t)$$

Also 
$$d(x_p)/dt = u_p, \quad d(y_p)/dt = v_p$$

### Point mass 

Point mass $m_p$ is used to solve point momentum $\mu_p = m_p \bf{u}_p$ and kinetic energy $K_p = m_p \frac{\bf{u}_p^2}{2}$

<img src="Ins.jpg" width = "200" height = "400" div align=center /> 

$$m_p = \sum_{z\in Z(p)} A_z^p \rho^p_z$$

$$\rho_z(t) = m_z/V_z(t)$$

In [2]:
function MassAndDens(mz::Array{Float64,1},x::Array{Float64,1},y::Array{Float64,1})
    
    Ic, = size(mz)
    Ip, = size(Tc)
    Asbz = zeros(Float64,4)
    ρ = zeros(Float64,Ic)
    mp = zeros(Float64,Ip)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ic
        xz[1:4] = x[Tp[i,1:4]]
        yz[1:4] = y[Tp[i,1:4]]
        Asbz = A_z(xz,yz)
        Az = sum(Asbz)
        ρ[i] = mz[i]/Az
        
        for j = 1:4
            ip = Tp[i,j]
            mp[ip] = mp[ip] + Asbz[j]*ρ[i]
        end
    end
   # @show mp
    return ρ,mp
    
   
end           
        

MassAndDens (generic function with 1 method)


$$ A_z^1 = \frac{5A_{41}+5A_{12}+A_{23}+A_{34}}{12}$$
$$ A_z^2 = \frac{A_{41}+5A_{12}+5A_{23}+A_{34}}{12}$$
$$ A_z^3 = \frac{A_{41}+A_{12}+5A_{23}+5A_{34}}{12}$$
$$ A_z^4 = \frac{5A_{41}+A_{12}+A_{23}+5A_{34}}{12}$$

In [3]:
function A_z(xz::Array{Float64,1},yz::Array{Float64,1}) 
    A =zeros(Float64,4)
    xc= sum(xz[i] for i in 1:4)/4
    yc= sum(yz[i] for i in 1:4)/4
    
    A41 = Area3(xz[4],xz[1],xc,yz[4],yz[1],yc)
    A12 = Area3(xz[1],xz[2],xc,yz[1],yz[2],yc)
    A23 = Area3(xz[2],xz[3],xc,yz[2],yz[3],yc)
    A34 = Area3(xz[3],xz[4],xc,yz[3],yz[4],yc)

    A[1] = (5A41+5A12+A23+A34)/12
    A[2] = (A41+5A12+5A23+A34)/12
    A[3] = (A41+A12+5A23+5A34)/12
    A[4] = (5A41+A12+A23+5A34)/12
    
    

    return A
end

A_z (generic function with 1 method)

<img src="n1234.png" width = "300" height = "300" div align=center />

$$\overrightarrow{n}_{12} = \frac{1}{L_{12}}(y_1- y_2, x_2-x_1)  $$

$$ \mathbf{F_1}  = \frac{1}{2} ( p \cdot \overrightarrow{n}_{12}L_{12} + p\cdot \overrightarrow{n}_{41}L_{41}) $$ 

$$ \mathbf{F_1}  = \frac{1}{2}  p \left[ (y_1 - y_2, x_2 -x_1) + (y_4 -y_1, x_1 - x_4) \right] $$ 

In [29]:
function F_z(x::Array{Float64,1},y::Array{Float64,1},p::Float64)
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
    
   # L12 = √((x[2]-x[1])^2+(y[2]-y[1])^2)
    n12x = -y[2]+y[1]
    n12y = x[2]-x[1]
    
  #  L41 = √((x[1]-x[4])^2+(y[1]-y[4])^2)
    n41x = -y[1]+y[4]
    n41y =  x[1]-x[4]
    
   # L23 = √((x[3]-x[2])^2+(y[3]-y[2])^2)
    n23x = -y[3]+y[2]
    n23y =  x[3]-x[2]
    
  #  L34 = √((x[4]-x[3])^2+(y[4]-y[3])^2)
    n34x = -y[4]+y[3]
    n34y =  x[4]-x[3]
    
    Fx[1] = p*(n12x+n41x)/2
    Fy[1] = p*(n12y+n41y)/2
    
    Fx[2] = p*(n12x+n23x)/2
    Fy[2] = p*(n12y+n23y)/2
    
    Fx[3] = p*(n23x+n34x)/2
    Fy[3] = p*(n23y+n34y)/2
    
    Fx[4] = p*(n34x+n41x)/2
    Fy[4] = p*(n34y+n41y)/2
    
    return Fx,Fy
end           

F_z (generic function with 1 method)

对于单元 
$$ \frac{d e}{d t} = -\frac{p}{\rho}\nabla \cdot \mathbf{u}$$ 
在单元上做积分

$$ m_z \frac{de_z}{dt} = \sum_{z\in Z(p)} \bf{F}_z^p \cdot \bf{u}_p $$  

对于节点
$$ \rho \frac{d \mathbf{u}}{dt} = - \nabla p$$

$$m_p \frac{d\bf{u}_p}{dt} = -\sum_{p\in P(z)} \bf{f}_z^p$$

In [67]:
function Rhs(x::Array{Float64,1},y::Array{Float64,1},u::Array{Float64,1},v::Array{Float64,1},p::Array{Float64,1})  #Right hand e
    Ic, = size(Tp)
    Ip, = size(Tc)
    rhsu = zeros(Float64,Ip)
    rhsv = zeros(Float64,Ip) 
    rhse = zeros(Float64,Ic)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    
    for i in 1:Ic
        for j =1:4
            xz[j] = x[Tp[i,j]]
            yz[j] = y[Tp[i,j]]
        end
        Fx,Fy = F_z(xz,yz,p[i])
        
        ∑=0
        for j in 1:4
            ip = Tp[i,j]
            ∑ = ∑ +u[ip]*Fx[j] + v[ip]*Fy[j]
        end
        rhse[i] = ∑
        
        for j = 1:4
            ip = Tp[i,j]
            rhsu[ip] +=  -Fx[j]
            rhsv[ip] +=  -Fy[j]
        end     
    end
    return rhse, rhsu,rhsv
end

Rhs (generic function with 1 method)

In [7]:
function Euler1st(dt,u,v,x,y,p,mz,ρ,α)
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,x,y)
   
 #   @show mp
    
    e = pToe(p, ρ,problem.γ)
    
    p = artificial_viscosity!(p,ρ,x,y,u,v,α)
    
    rhse, rhsu,rhsv = Rhs(x,y,u, v, p)
    
    rhsu,rhsv = Bound(rhsu,rhsv,x,y,u, v, p)
    
    x  +=  dt*u
    y  +=  dt*v
    u  += dt*(rhsu ./ mp)
    v  += dt*(rhsv ./ mp)
    e  += dt*(rhse ./ mz)
    

    ρ,mp = MassAndDens(mz,x,y)
    
    p = eTop(e, ρ,problem.γ)
    return u,v,x,y,p,ρ
end

Euler1st (generic function with 1 method)

In [70]:
function predictor_corrector2nd(dt,u,v,x,y,p,mz,ρ,α)
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,x,y)
    e = pToe(p, ρ,problem.γ)  
    p = artificial_viscosity!(p,ρ,x,y,u,v,α)
    
    rhse, rhsu,rhsv = Rhs(x,y,u, v, p)
    
    rhsu,rhsv = Bound(rhsu,rhsv,x,y,u, v, p)
    
    x₀ =  x + dt*u
    y₀  = y + dt*v
    u₀  = u + dt*(rhsu ./ mp)
    v₀  = v + dt*(rhsv ./ mp)
    e₀  = e + dt*(rhse ./ mz)
    

    ρ,mp = MassAndDens(mz,x₀,y₀)
    p = eTop(e₀, ρ,problem.γ)   
    
    p = artificial_viscosity!(p,ρ,x₀,y₀,u₀,v₀,α) 
    rhse₀,rhsu₀,rhsv₀ = Rhs(x₀,y₀,u₀, v₀, p)
    rhsu₀,rhsv₀ = Bound(rhsu₀,rhsv₀,x₀,y₀,u₀, v₀, p)
    
    x +=  0.5dt*(u + u₀)
    y +=  0.5dt*(v + v₀)
    u +=  0.5dt*(rhsu + rhsu₀) ./ mp
    v +=  0.5dt*(rhsv + rhsv₀) ./ mp
    e +=  0.5dt*(rhse + rhse₀)  ./ mz
    
    ρ,mp = MassAndDens(mz,x,y)
    p = eTop(e₀, ρ,problem.γ) 
    
    return u,v,x,y,p,ρ
end

predictor_corrector2nd (generic function with 1 method)

In [68]:
function Bound(rhsu,rhsv,x,y,u,v,p)
    Ip, = size(Tc)
    Ib, = size(Tb)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ib
        ip = Tb[i,1]
    #    @show ip
        for j = 1:4
            ic = Tc[ip,j]
            if ic != 0
                xz[1:4] = x[Tp[ic,1:4]]
                yz[1:4] = y[Tp[ic,1:4]]
                if Tb[i,2] ==1
                    Fx,Fy = F_zB(xz,yz,p[ic],j,1)
                    rhsu[ip] +=  Fx
                    rhsv[ip] += Fy
                   # mp[ip] *= 2.0
                end
                if Tb[i,3] ==1
                    Fx,Fy = F_zB(xz,yz,p[ic],j,2)
                    rhsu[ip] += Fx
                    rhsv[ip] += Fy
                   # mp[ip] *= 2.0
                end
            end
        end
    end
    return rhsu, rhsv
end  

Bound (generic function with 1 method)

### Boundary force

If one edge of a cell is a free boundary, then $f$ on the points is zero. As we have add it in RHs(), we need to remove it by resolve it again. Different from RHs(), at every point of a cell the force is composited by two parts on different edges. This is distinguished by $k$ when $k = 1$ the boundary is on y direction, forces on 12 and 34 boundary must be subtracted. When $ k =2$ the boundary is on x direction, subtract forces on 41, 23. 

In [11]:
function F_zB(x::Array{Float64,1},y::Array{Float64,1},p::Float64,j::Int,k::Int)
    Fx = 0.0
    Fy = 0.0      
    if k==1 # y direction 
     #   L12 = √((x[2]-x[1])^2+(y[2]-y[1])^2)
        n12x = -y[2]+y[1]
        n12y = x[2]-x[1]
     #   L34 = √((x[4]-x[3])^2+(y[4]-y[3])^2)
        n34x = -y[4]+y[3]
        n34y = x[4]-x[3]
        if j == 1  #Point 1
            Fx = p*n12x/2
            Fy = p*n12y/2
        elseif j == 2 
            Fx = p*n12x/2
            Fy = p*n12y/2
        elseif j == 3
            Fx = p*n34x/2
            Fy = p*n34y/2
        else 
            Fx = p*n34x/2
            Fy = p*n34y/2
        end
    elseif k == 2
      #  L41 = √((x[1]-x[4])^2+(y[1]-y[4])^2)
        n41x = -y[1]+y[4]
        n41y =  x[1]-x[4]

       # L23 = √((x[3]-x[2])^2+(y[3]-y[2])^2)
        n23x = -y[3]+y[2]
        n23y = x[3]-x[2]
        if j == 1
            Fx = p*n41x/2
            Fy = p*n41y/2
        elseif j == 2 
            Fx = p*n23x/2
            Fy = p*n23y/2
        elseif j == 3
            Fx = p*n23x/2
            Fy = p*n23y/2
        else 
            Fx = p*n41x/2
            Fy = p*n41y/2
        end
    end
    return Fx,Fy
    end           

F_zB (generic function with 1 method)

### EOS 
$$ e = \frac{p}{(\gamma-1)\rho}$$

In [12]:
function pToe(p,ρ,γ)
    return p ./ ρ/(γ-1)
end
function eTop(e,ρ,γ)
    return (γ-1)*ρ .* e
end

eTop (generic function with 1 method)

In [13]:
function CFL(SF,x::Array{Float64,1},y::Array{Float64,1},
                u::Array{Float64,1},v::Array{Float64},
                p::Array{Float64,1},ρ::Array{Float64,1})
    Ic, = size(ρ)
    cflmin = 1.e9
      cfl = 0 
    γ = problem.γ
    for i in 1:Ic
        cfl =0
        u2max = 0
        u2=0.0
        for j = 1:4
            ip = Tp[i,j]
            u2 = u[ip]^2+v[ip]^2
            if u2 > u2max ; u2max = u2; end
        end
        
        dlmin = (x[Tp[i,4]] - x[Tp[i,1]])^2+(y[Tp[i,4]]-y[Tp[i,1]])^2
        for j=1:3
            dl= (x[Tp[i,j+1]] - x[Tp[i,j]])^2+(y[Tp[i,j+1]]-y[Tp[i,j]])^2
            if dl < dlmin; dlmin=dl; end
        end
        c = √(γ*p[i]/ρ[i])
        cfl = √(dlmin)/(√(u2max)+c)
        if cfl < cflmin; cflmin = cfl; end
    end
 #   @show cflmin
    dt = cflmin*SF
    return dt
end 
        

CFL (generic function with 1 method)

In [72]:
function TimeSolve(u,v,x,y,p,mz,ρ,α)
    tt= problem.tt
    sf =problem.sf
    t= 0.0
    while t<tt
#for i in 1:50
        dt=CFL(sf,x,y,u,v,p,ρ)
        if t+dt>tt
           dt = tt-t
        end
        
        u,v,x,y,p,ρ = predictor_corrector2nd(dt,u,v,x,y,p,mz,ρ,α)
        t += dt
        
        println(t)
    end
#
    return u,v,x,y,p,ρ
end

TimeSolve (generic function with 1 method)

## Artificial Viscosity 

Wilkins（1980） use the strain rate along the direction of cell acceleration to construct the artificial viscosity 
$$\frac{ds}{dt} = \frac{\partial u}{\partial x} \text{cos}^2 \alpha + \frac{\partial v}{\partial y} \text{sin}^2\alpha +(\frac{\partial u}{\partial y} + \frac{\partial v}{\partial x})\text{sin} \alpha \text{cos} \alpha$$


Then the artificial viscosity can be written as
$$ q_w = \left\{ \begin{align}
l_0^2 \rho \left( \frac{ds}{dt} \right)^2 +  l_L\rho c \left|\frac{ds}{dt}\right|, \quad \text{if}  \quad \frac{ds}{dt}< 0,\\
0, \quad \text{if} \quad \frac{ds}{dt}\ge 0. \\
\end{align}
\right.
$$
where $l_0 = a_0L$, $l_L = a_L L$, $a_0 \approx 2$, $a_L \approx 1$ and $c$ is the sonic speed.

$L$ is the reference length, and constructed as
$$L = \frac{2A}{d_1 + d_2 +d_3 +d_4} $$


<img src="L1.png" width = "400" height = "300" div align=center />

In [82]:
function artificial_viscosity_c!(p,ρ,x,y,u,v,α)
    γ = problem.γ
    xc = sum(x[i] for i in 1:4)/4
    yc = sum(y[i] for i in 1:4)/4
    
    ∂u_∂x,∂u_∂y,∂v_∂x,∂v_∂y = ∂uv(x,y,u,v)
    
    ds_dt = ∂u_∂x*cosd(α)^2 +∂v_∂y*sind(α)^2 + (∂u_∂y+∂v_∂x)*sind(α)*cosd(α)
    
    L = l_viscosity(x,y,α)
    a₀ = 2; aₗ = 1
    c = √(γ*p/ρ)
    l₀ = a₀*L
    lₗ = aₗ*L
    
    L = 1/2*(√((x[3] - x[1])^2+(y[3] - y[1])^2) +√((x[2] - x[4])^2+(y[2] - y[4])^2) )  
    
   # if ds_dt > 0
    if ∂u_∂x > 0
        q = 0
    else
        #q = l₀^2*ρ*(ds_dt)^2 +lₗ*ρ*c*abs(ds_dt)
        q = 0.5ρ*L*c*abs(∂u_∂x) +0.06L^2*ρ*(∂u_∂x)^2 
    end
    return p+q
end

artificial_viscosity_c! (generic function with 1 method)

$$b_2 \rho Lc \frac{\partial u}{\partial x} - b_{12}\rho L^2 \left(\frac{\partial u}{\partial x}\right)^2$$

In [16]:
function artificial_viscosity!(p,ρ,x,y,u,v,α)
    I, = size(Tp)
    
    xc = zeros(Float64,4)
    yc = zeros(Float64,4)
    uc = zeros(Float64,4)
    vc = zeros(Float64,4)
    for i in 1:I
        for j in 1:4
            ip = Tp[i,j]
            xc[j] = x[ip]
            yc[j] = y[ip]
            uc[j] = u[ip]
            vc[j] = v[ip]
        end
        p[i] = artificial_viscosity_c!(p[i],ρ[i],xc,yc,uc,vc,α[i])
    end
    
        return p
        
    end       

artificial_viscosity! (generic function with 1 method)

In [17]:
function l_viscosity(x,y,α)
    xc = sum(x)/4; yc = sum(y)/4
    d = zeros(Float64,4)
    for i in 1:4
    d[i] = distance(x[i],y[i],xc,yc,α)
    end
    A = A_z(x, y)
    L = 2sum(A)/sum(d)
    return L
end

l_viscosity (generic function with 1 method)

#### distance from  Point to line 
Line: $$ (xc -cos \alpha )(x-xc) = (yc-sin \alpha)(y-yc)$$
point: $x,y$

In [18]:
function distance(x,y,xc,yc,α)
    d = abs((xc - cosd(α)) * (x .-xc)- (yc-sind(α))*(y .- yc))/√((xc-cosd(α))^2+(yc-sind(α))^2)
    return d
end

distance (generic function with 1 method)

In [19]:
struct prb
    tt::Float64
    sf::Float64
    γ ::Float64
end

In [20]:
function Area3(x1,x2,x3,y1,y2,y3)
    return abs((x1*y2+y1*x3+x2*y3-x1*y3-y1*x2-y2*x3)/2)
end

Area3 (generic function with 1 method)

###  $\frac{\partial (u,v)}{\partial (x,y)}$

we use the diffences between 1 and 3 points and 2 and 4 points to solve the partial differences approximately, as
$$ \frac{\partial u}{\partial x}  \approx \frac{1}{2}\frac{u_3-u_1}{x_3-x_1}+\frac{1}{2}\frac{u_4-u_2}{x_4-x_2}$$ 
Similar process to $\frac{\partial u}{\partial y}$,$\frac{\partial v}{\partial x}$ and $\frac{\partial v}{\partial y}$

In [78]:
function ∂uv(x,y,u,v)
    ∂u_∂x = 0.5(u[3]-u[1])/(x[3]-x[1]) + 0.5(u[2]-u[4])/(x[2]-x[4])
    ∂v_∂x = 0.5(v[3]-v[1])/(x[3]-x[1]) + 0.5(v[2]-v[4])/(x[2]-x[4])
    ∂u_∂y = 0.5(u[3]-u[1])/(y[3]-y[1]) + 0.5(u[4]-u[2])/(y[4]-y[2])
    ∂v_∂y = 0.5(v[3]-v[1])/(y[3]-y[1]) + 0.5(v[4]-v[2])/(y[4]-y[2])
    return ∂u_∂x,∂u_∂y,∂v_∂x,∂v_∂y
end

∂uv (generic function with 1 method)

### noh 算例 

In [57]:
@inline function InitTest1()

    global problem = prb(0.1,0.3,1.4)
     
    I = 200
    J = 20
    dx = 1.0/I
    dy = 0.1/J
    
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, I*2+J*2,3) #边界 
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:I 
        for j in 1:J
            ic = i+(j-1)*I
            Tp[ic,1] = i + (j-1)*(I+1)
            Tp[ic,2] = i+1 +(j-1)*(I+1)
            Tp[ic,3]=i+1+j*(I+1)
            Tp[ic,4]=i+j*(I+1) 
        end
    end
    
    for i = 1:I+1
        for j = 1:J+1
            ip = i+(j-1)*(I+1)
            Tc[ip,1] = i +(j-1)*I #(i,j)
            Tc[ip,2] = i-1+(j-1)*I #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I   #(i,j-1)
            end
        end
    

# Bound 
  ib = 0
    for i = 1:I+1
        for j = 1:J+1
            if i == 1 || i== I+1 ||  j == 1 || j == J+1
                ib += 1
                ip =i+(j-1)*(I+1) 
                Tb[ib,1] = ip
                if i==1 
                   Tb[ib,3] = 1
                    Tc[ip,2] =0
                    Tc[ip,3] =0
                end   
                 if  i==I+1 
                    Tb[ib,3] = 1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
                    Tc[ip,1] =0
                    Tc[ip,4] =0
                end
            
                if j==1 
                    Tb[ib,2] = 1
                    Tc[ip,3] = 0
                    Tc[ip,4] = 0
                end
                if j==J+1
                    Tb[ib,2] = 1
                    Tc[ip,1] = 0
                    Tc[ip,2] = 0
                end
            end
        end
    end

    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    α = zeros(Float64,Ic)
    for i in 1:I+1
        for j = 1:J+1
            ip =i+(j-1)*(I+1)
            x[ip] = i*dx
            y[ip] = j*dy
            
            u[ip] = 1.0
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I
        for j in 1:J
            ic = i+(j-1)*I
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A) #[1]+A[2]+A[3]+A[4]
          #  if i<=100
                mz[ic] = (2+sin(2π*dx*i)) * Az[ic]
                p[ic] = 1.0
                ρ[ic] = sin(2π*dx*i)+2
#             else
#                 mz[ic] = 0.1 * Az[ic]
#                 p[ic] = 1.0
#                 ρ[ic] = 0.1
#             end
        end
    end
    
    return x,y,u,v,mz,p,ρ,Az,α
end

InitTest1 (generic function with 1 method)

###  1D sod 算例

In [62]:
@inline function InitTest2()

    global problem = prb(0.13,0.1,1.4)
     
    I = 300
    J = 10
    dx = 1.0/I
    dy = 0.1/J
    
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, I*2+J*2,3) #边界 
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:I 
        for j in 1:J
            ic = i+(j-1)*I
            Tp[ic,1] = i + (j-1)*(I+1)
            Tp[ic,2] = i+1 +(j-1)*(I+1)
            Tp[ic,3]=i+1+j*(I+1)
            Tp[ic,4]=i+j*(I+1) 
        end
    end
    
    for i = 1:I+1
        for j = 1:J+1
            ip = i+(j-1)*(I+1)
            Tc[ip,1] = i +(j-1)*I #(i,j)
            Tc[ip,2] = i-1+(j-1)*I #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I   #(i,j-1)
            end
        end
    

# Bound 
  ib = 0
    for i = 1:I+1
        for j = 1:J+1
            if i == 1 || i== I+1 ||  j == 1 || j == J+1
                ib += 1
                ip =i+(j-1)*(I+1) 
                Tb[ib,1] = ip
                if i==1 
                   Tb[ib,3] = 1
                    Tc[ip,2] =0
                    Tc[ip,3] =0
                end   
                 if  i==I+1 
                    Tb[ib,3] = 1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
                    Tc[ip,1] =0
                    Tc[ip,4] =0
                end
            
                if j==1 
                    Tb[ib,2] = 1
                    Tc[ip,3] = 0
                    Tc[ip,4] = 0
                end
                if j==J+1
                    Tb[ib,2] = 1
                    Tc[ip,1] = 0
                    Tc[ip,2] = 0
                end
            end
        end
    end

    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    α = zeros(Float64,Ic)
    for i in 1:I+1
        for j = 1:J+1
            ip =i+(j-1)*(I+1)
            x[ip] = i*dx
            y[ip] = j*dy
            if i >=I/2
                u[ip] = 0.
            else
                u[ip] = 0
            end
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I
        for j in 1:J
            ic = i+(j-1)*I
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A) #[1]+A[2]+A[3]+A[4]
            if i>=I/2
                mz[ic] = 0.125 * Az[ic]
                p[ic] = 0.1
                ρ[ic] = 0.125
            else
                mz[ic] = 1.0 * Az[ic]
                p[ic] = 1.0
                ρ[ic] = 1.0
            end
        end
    end
    
    return x,y,u,v,mz,p,ρ,Az,α,I,J
end

InitTest2 (generic function with 1 method)

In [83]:
x,y,u,v,mz,p,ρ,Az,α,Ix,Iy=InitTest2()
#Output1
u,v,x,y,p,ρ = TimeSolve(u, v, x, y, p, mz,ρ,α)
Output(x, y, u, v, p, ρ, mz)

0.00028171808490950264
0.0005366478204066392
0.0007752487546020774
0.0010035708734184327
0.001213268688164802
0.0014079409729515341
0.0015902361637853846
0.0017621650828470543
0.0019252917895114122
0.002080856822979997
0.0022298602684221627
0.0023731196608604965
0.0025113114614423123
0.00264500144095416
0.002774667361514883
0.002900716184681444
0.003023497311854281
0.0031433128993030575
0.003260425984210179
0.0033750669512939346
0.0034874387267577003
0.0035977209858736006
0.003706073588728241
0.00381263940662625
0.00391754666345422
0.00402091088795608
0.004122836551610912
0.004223418450714157
0.004322742878981753
0.0044208886275480435
0.0045179278419044825
0.004613926759609144
0.004708946348104117
0.004803042858423928
0.004896268307749385
0.004988670901496086
0.005080295403802547
0.005171183463805385
0.005261373903885906
0.0053509029750878935
0.005439804584096622
0.005528110495500018
0.005615850512497661
0.005703052638760471
0.005789743223756734
0.005875947093534857
0.00596168766867898

0.0331438111718912
0.033219205146866854
0.033294589214093416
0.03336996347326528
0.03344532802337227
0.03352068296271195
0.03359602838890017
0.03367136439888017
0.03374669108893007
0.03382200855466907
0.03389731689106232
0.033972616192424615
0.03404790655242298
0.03412318806407824
0.03419846081976573
0.03427372491121505
0.034348980429509214
0.03442422709674604
0.03449946121103894
0.034574682906086805
0.03464989231433369
0.03472508956698066
0.034800274793996916
0.034875448124130286
0.03495060968491695
0.03502575960269061
0.035100898002591106
0.03517602500857248
0.0352511407434106
0.035326245328710267
0.03540133888491204
0.0354764215312986
0.03555149338600086
0.035626554566003686
0.03570160518715147
0.03577664536415339
0.03585167521058849
0.03592669483891061
0.03600170436045309
0.03607670388543347
0.036151693522957926
0.036226673381025774
0.036301643566533814
0.036376604185280645
0.03645155534197099
0.036526497140219966
0.03660142968255737
0.03667635307043199
0.03675126740421595
0.036826

0.0633110158986369
0.06338401795231377
0.06345701716202476
0.06353001355240306
0.06360300714784868
0.0636759979725307
0.06374898605038956
0.06382197140513925
0.0638949540602695
0.06396793403904803
0.06404091136452263
0.06411388605952331
0.06418685814666444
0.06425982764834684
0.06433279458675983
0.0644057589838833
0.06447872086148972
0.06455168024114614
0.06462463714421626
0.0646975915918623
0.06477054360504697
0.06484349320453547
0.06491644041089734
0.06498938524450834
0.0650623277255524
0.06513526787402338
0.06520820570972698
0.06528114122119993
0.06535407372455151
0.06542700324578975
0.06549992981067886
0.06557285344474156
0.06564577417326141
0.06571869202128507
0.06579160701362462
0.06586451917485983
0.06593742852934036
0.06601033510118796
0.06608323891429874
0.06615613999234521
0.06622903835877855
0.06630193403683067
0.0663748270495163
0.06644771741963516
0.06652060516977393
0.06659349032230832
0.06666637289940515
0.06673925292302427
0.06681213041492061
0.06688500539664612
0.06695

0.0935054171907338
0.09357769692665546
0.09364997568696044
0.09372225347893569
0.09379453030980653
0.09386680618673719
0.09393908111683137
0.09401135510713271
0.09408362816462533
0.09415590029623438
0.09422817150882651
0.09430044179026362
0.09437271093295085
0.0944449789455417
0.09451724583661662
0.09458951161468372
0.09466177628817934
0.09473403986546867
0.09480630235484638
0.09487856376453722
0.09495082410269659
0.0950230833774112
0.09509534159669962
0.0951675987685129
0.09523985490073512
0.09531211000118399
0.09538436407761137
0.09545661713770393
0.0955288691890836
0.09560112023930821
0.09567337029587202
0.09574561936620624
0.0958178674576796
0.0958901145775989
0.0959623607332095
0.0960346059316959
0.0961068501801822
0.09617909348573268
0.0962513358553523
0.09632357729598717
0.09639581781452514
0.09646805741779618
0.096540296112573
0.09661253390557138
0.09668477080345055
0.09675700681281418
0.0968292419402106
0.09690147619213323
0.09697370957502106
0.09704594209525916
0.097118173759

0.12343159232319907
0.12350357141606728
0.12357554998898676
0.12364752804550601
0.1237195055891456
0.12379148262339836
0.1238634591517296
0.12393543517757735
0.1240074107043526
0.12407938573543946
0.12415136027419543
0.12422333432395159
0.12429530788801282
0.124367280969658
0.12443925357214025
0.12451122569868711
0.12458319735250076
0.1246551685367582
0.12472713925460413
0.12479910950916541
0.12487107930354463
0.12494304864082004
0.12501501752404456
0.1250869859562459
0.12515895394043033
0.12523092147958056
0.12530288857665586
0.12537485523459235
0.1254468214563031
0.12551878724467835
0.12559075260258576
0.12566271749779834
0.12573468187643771
0.12580664574202796
0.12587860909806547
0.12595057194801923
0.12602253429533097
0.12609449614341545
0.1261664574956606
0.1262384183554279
0.12631037872605236
0.12638233861084291
0.12645429801308256
0.12652625693602854
0.12659821538290922
0.1266701733569256
0.12674213086125882
0.12681408789906515
0.12688604447347612
0.12695800058759882
0.127029956

In [60]:
using DelimitedFiles
function Output(x,y,u,v,p,ρ,mz)

    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data.dat", "w+")
    A = zeros(Float64,Ip,6)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"u\" \"v\" \"rho\" \"p\"  ZONE T=\"Zone 1\" 
I=301,J=11,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)

    for i in 1:Ip
        nc = 0
        ρz = 0.0
        pz = 0.0
        for j = 1:4
            ic = Tc[i,j] 
            if ic != 0
                ρz += ρ[ic]
                pz += p[ic]
                nc +=  1
            end
        end
        ρz = ρz/nc
        pz = pz/nc
        A[i,1] = x[i]
        A[i,2] = y[i]
        A[i,3] = u[i]
        A[i,4] = v[i]
        A[i,5] = ρz
        A[i,6] = pz
        
    end
    writedlm(io, A, "  ")
    close(io)
end

Output (generic function with 1 method)

In [27]:
using DelimitedFiles
function Output1(x,y,rhs)

    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data1.dat", "w+")
    A = zeros(Float64,Ip,3)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"p\"  ZONE T=\"Zone 1\" 
I=400,J=40,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)

    for i in 1:Ic
        nc = 0
        ρz = 0.0
        pz = 0.0
        ip  = Tp[i,1]
        A[i,1] = x[ip]
        A[i,2] = y[ip]
        A[i,3] = rhs[i]
        
    end
    writedlm(io, A, "  ")
    close(io)
end

Output1 (generic function with 1 method)

In [24]:
using DelimitedFiles
function Output2(x,y,rhs)

    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data1.dat", "w+")
    A = zeros(Float64,Ip,3)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"p\"  ZONE T=\"Zone 1\" 
I=401,J=41,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)

    for i in 1:Ip
        nc = 0
        ρz = 0.0
        pz = 0.0
       # ip  = Tp[i,1]
        A[i,1] = x[i]
        A[i,2] = y[i]
        A[i,3] = rhs[i]
        
    end
    writedlm(io, A, "  ")
    close(io)
end

Output2 (generic function with 1 method)

In [ ]:
function outputline(Ix,Iy,x,y,u,v,ρ,p,mz)
    I, = size(ρ)
    
    ip =i+(j-1)*(I+1)
    ic =i+(j-1)*I
    uL =  zeros(Float64,Ix)
    xL =  zeros(Float64,Ix)
    vL =  zeros(Float64,Ix)
    ρL =  zeros(Float64,Ix)        
    pL =  zeros(Float64,Ix)  
            
    for i in 1:Ix
        j = Int8(floor(Ij/2))
            ip = i+(j-1 ) *(I+1)
            ic =i+(j-1)*I
            uL[i] = u[ip]
            xL[i]  = x[ip]
            vL[i] = v[ip]
            ρL[i]  = ρ[ic]
            pL[i] = p[ic]
    end

In [61]:
Output(x, y, u, v, p, ρ, mz)

In [97]:
floor(0.3)

0.0

### 对流涡

### chenjuan shuqiwang 

In [74]:
using Plots   

In [75]:
scatter3d(x, y,ρ)

DimensionMismatch: DimensionMismatch("dimensions must match")

In [101]:
- 0.9/0.55*400

-654.5454545454545